This is MODIFIED code originally copied directly from https://github.com/FEMA/openfema-samples/blob/master/code-samples/api_paging_jsonoutput.py

Which is the github repo of working code samples provided by OpenFEMA https://www.fema.gov/about/openfema/developer-resources#paging

see also OpenFEMA API documentation https://www.fema.gov/about/openfema/api#filter

# LAST MODIFIED 1/9/2023 TO REMOVE COMMENTS AND ANY PARTS OF THE CODE THAT AREN'T GOING IN THE PAPER. SEE DRAFT 2 VERSION FOR THE FILE THAT IS BEST COMMENTED AND CLOSEST TO THE ORIGINAL

In [1]:
#!/usr/bin/env python3
# Paging example using Python 3. Output in JSON.
# update the code to the get CA claims from NFIP

import sys
import urllib.request
import json
import math
from datetime import datetime

# Base URL for this endpoint. Add filters, column selection, and sort order to this.
baseUrl = "https://www.fema.gov/api/open/v1/FimaNfipClaims?$filter=state%20eq%20%27CA%27&"

top = 5000      # number of records to get per call
skip = 0        # number of records to skip

# Return 1 record with your criteria to get total record count. 
# Specify only 1 column here to reduce amount of data returned.
# Need inlinecount to get record count.
webUrl = urllib.request.urlopen(
    baseUrl + "$inlinecount=allpages&$select=id&$top=1")
result = webUrl.read()
jsonData = json.loads(result.decode())

# calculate the number of calls we will need to get all of our data (using the maximum of 5000)
recCount = jsonData['metadata']['count']
loopNum = math.ceil(recCount / top)
#loopNum = 1

# send some logging info to the console so we know what is happening
print("START " + str(datetime.now()) + ", " + str(recCount) + " records, " +
      str(top) + " returned per call, " + str(loopNum) + " iterations needed.")

# Initialize our file. Only doing this because of the type of file wanted. See the loop below.
# The root json entity is usually the name of the dataset, but you can use any name.
# Set a new name each time the script is run because it keeps adding to the old file
outFile = open("output4.json", "a")
outFile.write('{"FimaNfipClaims":[')

# Loop and call the API endpoint changing the record start each iteration. 
# The metadata is being suppressed as we no longer need it.
i = 0
while (i < loopNum):
    # By default data is returned as a JSON object, the data set name being the root element. Unless
    #   you extract records as you process, you will end up with 1 distinct JSON object for EVERY
    #   call/iteration. An alternative is to return the data as JSONA (an array of json objects) with
    #   no root element - just a bracket at the start and end. This is easier to manipulate.
    webUrl = urllib.request.urlopen(
        baseUrl + "&$metadata=off&$format=jsona&$skip=" + str(skip) + "&$top=" + str(top))
    result = webUrl.read()

    # The data is already returned in a JSON format. There is no need to decode and load as a JSON object.
    #   If you want to begin working with and manipulating the JSON, import the json library and load with
    #   something like: jsonData = json.loads(result.decode())

    # Append results to file, trimming off first and last JSONA brackets, adding comma except for last call,
    #   AND root element terminating array bracket and brace to end unless on last call. The goal here is to
    #   create a valid JSON file that contains ALL the records. This can be done differently.
    if (i == (loopNum - 1)):
        # on the last so terminate the single JSON object
        outFile.write(str(result[1:-1], 'utf-8') + "]}")
    else:
        outFile.write(str(result[1:-1], 'utf-8') + ",")

    # increment the loop counter and skip value
    i += 1
    skip = i * top

    print("Iteration " + str(i) + " done")

outFile.close()


START 2023-01-09 16:46:08.710367, 50994 records, 5000 returned per call, 11 iterations needed.
Iteration 1 done
Iteration 2 done
Iteration 3 done
Iteration 4 done
Iteration 5 done
Iteration 6 done
Iteration 7 done
Iteration 8 done
Iteration 9 done
Iteration 10 done
Iteration 11 done


In [2]:
# lets re-open the file and see if we got the number of records we expected
inFile = open("output4.json", "r")
my_data = json.load(inFile)
print("END " + str(datetime.now()) + ", " + str(len(my_data['FimaNfipClaims'])) + " records in file")
inFile.close()

END 2023-01-09 16:47:45.967360, 50994 records in file


# THIS NEXT PART IS HOW TO GET THE RESULTS READ FROM THE WEBPAGE EXPORTED AS A .CSV
# YOU CAN ALSO READ THE .JSON DIRECTLY FROM THE OUTPUT FILE INTO A DATAFRAME, WHICH IS HOW IT IS DONE IN MY PAPER

In [10]:
#[naomi says: this loads the results just returned from the website, not the file that was written]

import json

jsonData = json.loads(result.decode())

In [12]:
#there are two keys, the first is the file name, the second is the one with the data
print(jsonData[:1])

[{'agricultureStructureIndicator': False, 'asOfDate': '2022-02-26T00:00:00.000Z', 'baseFloodElevation': None, 'basementEnclosureCrawlspace': 0, 'reportedCity': 'Temporarily Unavailable', 'condominiumIndicator': 'N', 'policyCount': 1, 'countyCode': '06097', 'communityRatingSystemDiscount': 10, 'dateOfLoss': '2019-02-26T00:00:00.000Z', 'elevatedBuildingIndicator': False, 'elevationCertificateIndicator': 3, 'elevationDifference': None, 'censusTract': '06097153705', 'floodZone': 'AE', 'houseWorship': False, 'latitude': '38.5', 'longitude': '-122.9', 'locationOfContents': 3, 'lowestAdjacentGrade': None, 'lowestFloorElevation': 0, 'numberOfFloorsInTheInsuredBuilding': 1, 'nonProfitIndicator': False, 'obstructionType': None, 'occupancyType': 6, 'originalConstructionDate': '1960-01-01T00:00:00.000Z', 'originalNBDate': '2000-01-01T00:00:00.000Z', 'amountPaidOnBuildingClaim': 221485.01, 'amountPaidOnContentsClaim': 49423.76, 'amountPaidOnIncreasedCostOfComplianceClaim': None, 'postFIRMConstructi

In [14]:
import pandas as pd

In [15]:
df = pd.DataFrame(jsonData)
df.head()

,agricultureStructureIndicator,asOfDate,baseFloodElevation,basementEnclosureCrawlspace,reportedCity,condominiumIndicator,policyCount,countyCode,communityRatingSystemDiscount,dateOfLoss,...,postFIRMConstructionIndicator,rateMethod,smallBusinessIndicatorBuilding,state,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,yearOfLoss,reportedZipcode,primaryResidence,id
0,False,2022-02-26T00:00:00.000Z,NaN,0.0,Temporarily Unavailable,N,1,06097,10.0,2019-02-26T00:00:00.000Z,...,False,1,False,CA,500000,100000,2019,95436,False,3c92bad1-5db5-48c3-9ee6-4559bdc0cb6e
1,False,2022-02-26T00:00:00.000Z,77.0,0.0,Temporarily Unavailable,N,1,06097,10.0,2019-02-26T00:00:00.000Z,...,True,1,False,CA,100000,50000,2019,95436,False,7d6dde0e-6f73-4834-a6de-78a1e076a0f0
2,False,2022-02-26T00:00:00.000Z,NaN,NaN,Temporarily Unavailable,N,1,06067,NaN,1995-01-10T00:00:00.000Z,...,False,3,False,CA,62000,26900,1995,95641,False,f7f84dac-a63a-40c1-910d-b254bae24cae
3,False,2022-02-26T00:00:00.000Z,NaN,NaN,Temporarily Unavailable,N,1,06067,NaN,1997-01-03T00:00:00.000Z,...,False,3,False,CA,75000,29600,1997,95641,False,c84d7984-da02-42b2-8638-f9c7de49258d
4,False,2022-02-26T00:00:00.000Z,NaN,0.0,Temporarily Unavailable,N,1,06037,NaN,2004-12-28T00:00:00.000Z,...,False,7,False,CA,250000,60000,2004,91301,True,68bf0af0-8d5c-4b41-90a4-10eb073f3334


In [12]:
df.columns

Index(['agricultureStructureIndicator', 'asOfDate', 'baseFloodElevation',
       'basementEnclosureCrawlspace', 'reportedCity', 'condominiumIndicator',
       'policyCount', 'countyCode', 'communityRatingSystemDiscount',
       'dateOfLoss', 'elevatedBuildingIndicator',
       'elevationCertificateIndicator', 'elevationDifference', 'censusTract',
       'floodZone', 'houseWorship', 'latitude', 'longitude',
       'locationOfContents', 'lowestAdjacentGrade', 'lowestFloorElevation',
       'numberOfFloorsInTheInsuredBuilding', 'nonProfitIndicator',
       'obstructionType', 'occupancyType', 'originalConstructionDate',
       'originalNBDate', 'amountPaidOnBuildingClaim',
       'amountPaidOnContentsClaim',
       'amountPaidOnIncreasedCostOfComplianceClaim',
       'postFIRMConstructionIndicator', 'rateMethod',
       'smallBusinessIndicatorBuilding', 'state',
       'totalBuildingInsuranceCoverage', 'totalContentsInsuranceCoverage',
       'yearOfLoss', 'reportedZipcode', 'primaryReside

In [16]:
df.describe()

,baseFloodElevation,basementEnclosureCrawlspace,policyCount,communityRatingSystemDiscount,elevationCertificateIndicator,elevationDifference,locationOfContents,lowestAdjacentGrade,lowestFloorElevation,numberOfFloorsInTheInsuredBuilding,occupancyType,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidOnIncreasedCostOfComplianceClaim,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,yearOfLoss
count,107.000000,400.000000,994.000000,362.000000,154.000000,110.000000,994.000000,96.000000,994.000000,994.000000,994.000000,659.000000,234.000000,37.000000,9.940000e+02,994.000000,994.000000
mean,331.467290,0.725000,1.324950,6.914365,1.837662,0.700000,2.096579,159.798958,35.755131,1.629779,2.609658,26055.334992,9075.001752,810.810811,2.049205e+05,39128.169014,2004.419517
std,1028.155918,1.281984,3.944739,1.747452,0.881701,4.852976,1.929975,307.005357,351.464033,0.824938,2.909952,57259.604916,21948.995495,4931.969619,7.918042e+05,67317.651434,13.839392
min,0.000000,0.000000,1.000000,1.000000,1.000000,-15.000000,0.000000,-0.500000,-5.000000,1.000000,1.000000,-5387.050000,-1156.040000,0.000000,0.000000e+00,0.000000,1977.000000
25%,11.000000,0.000000,1.000000,6.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2400.225000,406.390000,0.000000,5.200000e+04,0.000000,1995.000000
50%,53.000000,0.000000,1.000000,7.000000,2.000000,1.000000,3.000000,33.250000,0.000000,1.000000,1.000000,8923.900000,2340.905000,0.000000,1.467000e+05,10000.000000,2005.000000
75%,256.000000,1.000000,1.000000,7.000000,2.000000,2.000000,3.000000,95.550000,0.000000,2.000000,3.000000,24783.060000,10000.000000,0.000000,2.500000e+05,60000.000000,2017.000000
max,9990.000000,4.000000,103.000000,10.000000,4.000000,22.000000,7.000000,1329.100000,9990.000000,6.000000,18.000000,839906.060000,250000.000000,30000.000000,2.265680e+07,500000.000000,2022.000000


In [17]:
df.to_csv('ca_claims2.csv')